## 11-pyspark-pivot-unpivot-2.py

In [0]:
# 11-pyspark-pivot-unpivot-2.py
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
spark = SparkSession.builder.appName('PySparkExamples').getOrCreate()

In [0]:
data = [("Banana", 1000, "USA"), ("Carrots", 1500, "USA"), ("Beans", 1600, "USA"), \
        ("Orange", 2000, "USA"), ("Orange", 2000, "USA"), ("Banana", 400, "China"), \
        ("Carrots", 1200, "China"), ("Beans", 1500, "China"), ("Orange", 4000, "China"), \
        ("Banana", 2000, "Canada"), ("Carrots", 2000, "Canada"), ("Beans", 2000, "Mexico")]

columns = ["Product", "Amount", "Country"]
df = spark.createDataFrame(data = data, schema = columns)
print("DataFrame columns are:", df.columns, "with column count:", len(df.columns), "and with row count:", df.count())
df.printSchema()
df.show(truncate = False)

DataFrame columns are: ['Product', 'Amount', 'Country'] with column count: 3 and with row count: 12
root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



In [0]:
# Performing pivot operation
pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.printSchema()
pivotDF.show(truncate = False)

root
 |-- Product: string (nullable = true)
 |-- Canada: long (nullable = true)
 |-- China: long (nullable = true)
 |-- Mexico: long (nullable = true)
 |-- USA: long (nullable = true)

+-------+------+-----+------+----+
|Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
|Orange |null  |4000 |null  |4000|
|Beans  |null  |1500 |2000  |1600|
|Banana |2000  |400  |null  |1000|
|Carrots|2000  |1200 |null  |1500|
+-------+------+-----+------+----+



In [0]:
# Performing pivot operation
pivotDF = df.groupBy("Product","Country") \
            .sum("Amount") \
            .groupBy("Product") \
            .pivot("Country") \
            .sum("sum(Amount)")
pivotDF.printSchema()
pivotDF.show(truncate = False)

root
 |-- Product: string (nullable = true)
 |-- Canada: long (nullable = true)
 |-- China: long (nullable = true)
 |-- Mexico: long (nullable = true)
 |-- USA: long (nullable = true)

+-------+------+-----+------+----+
|Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
|Orange |null  |4000 |null  |4000|
|Beans  |null  |1500 |2000  |1600|
|Banana |2000  |400  |null  |1000|
|Carrots|2000  |1200 |null  |1500|
+-------+------+-----+------+----+



In [0]:
# Performing unpivot operation
unpivotExpr = "stack(3, 'Country Canada', Canada, 'Country China', China, 'Country Mexico', Mexico) as (Country, Total)"
unPivotDF = pivotDF.select("Product", expr(unpivotExpr)) \
            .where("Total is not null")
unPivotDF.show(truncate=False)

+-------+--------------+-----+
|Product|Country       |Total|
+-------+--------------+-----+
|Orange |Country China |4000 |
|Beans  |Country China |1500 |
|Beans  |Country Mexico|2000 |
|Banana |Country Canada|2000 |
|Banana |Country China |400  |
|Carrots|Country Canada|2000 |
|Carrots|Country China |1200 |
+-------+--------------+-----+

